In [3]:
import config as cf
from DISClib.ADT import list as lt
from DISClib.ADT import map as mp
from DISClib.ADT import graph as gph
from DISClib.DataStructures import mapentry as me
from DISClib.Algorithms.Sorting import shellsort as sa
import csv
assert cf

def create_catalog():
    catalog = {
        "routesdg" : gph.newGraph(directed=True, size=10000),
        "routesndg": gph.newGraph(size=3000),
        # "name2IATA" : mp.newMap(10000,maptype="PROBING"),
        "IATA2name" : mp.newMap(10000,maptype="PROBING"),
        "DuplicateRoute" : mp.newMap(50000,maptype="PROBING"),
        "Cities": mp.newMap(41002,maptype="PROBING")
    }

    return catalog

# Funciones para agregar informacion al catalogo

def add_airport(catalog, airport):
    table = catalog["IATA2name"]
    IATA = airport["IATA"]
    if not mp.contains(table,IATA):
        mp.put(table, IATA, airport)

catalog = create_catalog()

In [5]:
file = r"C:\Users\camil\OneDrive\Desktop\Los Andes\5to Semestre\EDA\Retos\Reto4-G08\Data\Skylines\airports_full.csv"
airports = csv.DictReader(open(file, encoding="utf-8"), delimiter=",")
for airport in airports:
    add_airport(catalog, airport)

In [12]:
a = set()

file = r"C:\Users\camil\OneDrive\Desktop\Los Andes\5to Semestre\EDA\Retos\Reto4-G08\Data\Skylines\routes_full.csv"
routes = csv.DictReader(open(file, encoding="utf-8"), delimiter=",")
for route in routes:
    a.add(route["Departure"])
    a.add(route["Destination"])

In [14]:
len(a)

3971